In [2]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

ImportError: No module named matplotlib.pyplot

In [13]:
import pandas as pd
import numpy as np 

In [18]:
df = c.data.make_base_df()

desc = 'gray_'
df = c.processing.save_grayscale(df)

AttributeError: 'module' object has no attribute 'data'

In [ ]:
test_df = c.data.make_test_df()
test_df = c.processing.save_grayscale(df)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC

vecs, types = c.data.random_forest_transform(df)
test_vecs, ids = c.random_forest_transform(test_df, True)
forest = RFC(n_jobs=2,n_estimators=50)

forest.fit(vecs, types)

In [ ]:
preds_probs = forest.predict_proba(test_vecs)

In [ ]:
c.submission.write_submission_file('output.csv', ids, preds_probs)